Projet de Machine Learning : expliquer le prix de l'électricité

Romain Foucher
Matthieu Evuort
Julie Chen


Introduction :

Le machine learning est une technologie qui vise à apprendre aux machines à tirer des enseignements des données et à s'améliorer avec l'expérience, au lieu d'être explicitement programmées pour le faire.

Ce projet consiste a expliquer le prix de l'électricité en France et en Allemagne selon une multitude de facteurs. On peut citer le climat, le réchauffement climatique, les  événements géopolitiques... De plus, chaque pays s’appuie sur un mix  énergétique qui lui est propre (nucléaire, solaire, hydrolique, gaz, charbon, etc).

 Pour cela, nous allons utiliser plusieurs méthodes afin de préparer nos données, de les analyser et de les modéliser grâce aux cours et aux TP que nous avons pu effectuer en cours mais ainsi qu'à la documantation d'internet.

Sommaire :

A) Préparation des données
B) Analyse exploratoire des données
C) Modélisation des données


A) Préparation des données

Afin de préparer nos données nous allons remplir les cases vides par la moyenne de la colonne. En effet, si nous supprimons les lignes ou les colonnes, nous perdrons des informations et nous réduisons la taille de l'échantillon. Cependant, nous n'allons pas supprimer ou remplacer les données "0.0" car elles sont utiles. En effet, il est possible que la variation du prix du charbon ou du gaz d'un pays ne change pas d'une journée à l'autre.

In [1]:
import sns as sns

from Data_preparation import data_x
for col in data_x.columns[data_x.isnull().any()]:
    data_x[col].fillna(data_x[col].mean(), inplace=True)

        ID  DAY_ID COUNTRY  DE_CONSUMPTION  FR_CONSUMPTION  DE_FR_EXCHANGE   
0     1054     206      FR        0.210099       -0.427458       -0.606523  \
1     2049     501      FR       -0.022399       -1.003452       -0.022063   
2     1924     687      FR        1.395035        1.978665        1.021305   
3      297     720      DE       -0.983324       -0.849198       -0.839586   
4     1101     818      FR        0.143807       -0.617038       -0.924990   
...    ...     ...     ...             ...             ...             ...   
1489   459     809      DE        1.529204        1.106682       -1.855327   
1490  1674     887      FR        1.618582        1.752840        0.611392   
1491   748    1083      DE        0.856399        0.489199       -0.255778   
1492  1454    1133      FR        0.560689       -0.343777       -0.830239   
1493   359    1118      DE        0.226730       -0.711005        0.873016   

      FR_DE_EXCHANGE  DE_NET_EXPORT  FR_NET_EXPORT  DE_NET_IMPO

Ensuite, nous fusionnons data_x et data_y pour comparer nos valeurs d'entrées avec nos valeurs de sorties en joignant sur l'ID. Puis, nous trions sur DAY_ID et COUNTRY pour une meilleure lisibilité.

In [2]:
from Data_preparation import data_x, data_y, pd
merged_X_Y = pd.merge(data_x, data_y, on='ID', how='inner')
merged_X_Y = merged_X_Y.sort_values(by=['DAY_ID', 'COUNTRY'])


Puisque nous étudions la France et l'Allemagne nous créons deux datas : data_FR et data_DE et nous ajoutons leurs colonnes respectives sans oublier de mettre TARGET.

In [3]:
data_FR = merged_X_Y.loc[merged_X_Y['COUNTRY'] == 'FR']
data_DE = merged_X_Y.loc[merged_X_Y['COUNTRY'] == 'DE']
data_FR = data_FR.filter(regex='FR|GAS_RET|COAL_RET|TARGET', axis=1)
data_DE = data_DE.filter(regex='DE|GAS_RET|COAL_RET|TARGET', axis=1)

Les valeurs des différents attributs sont comparables car ce sont toutes des variations dans le temps.

B) Analyse exploratoire des données

Afin de faire un aperçu des données nous utilisons plusieurs fonctions :


In [4]:
data_FR.info()
data_FR.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 851 entries, 742 to 1437
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   FR_CONSUMPTION    851 non-null    float64
 1   DE_FR_EXCHANGE    851 non-null    float64
 2   FR_DE_EXCHANGE    851 non-null    float64
 3   FR_NET_EXPORT     851 non-null    float64
 4   FR_NET_IMPORT     851 non-null    float64
 5   FR_GAS            851 non-null    float64
 6   FR_COAL           851 non-null    float64
 7   FR_HYDRO          851 non-null    float64
 8   FR_NUCLEAR        851 non-null    float64
 9   FR_SOLAR          851 non-null    float64
 10  FR_WINDPOW        851 non-null    float64
 11  FR_RESIDUAL_LOAD  851 non-null    float64
 12  FR_RAIN           851 non-null    float64
 13  FR_WIND           851 non-null    float64
 14  FR_TEMP           851 non-null    float64
 15  GAS_RET           851 non-null    float64
 16  COAL_RET          851 non-null    float64
 17 

,FR_CONSUMPTION,DE_FR_EXCHANGE,FR_DE_EXCHANGE,FR_NET_EXPORT,FR_NET_IMPORT,FR_GAS,FR_COAL,FR_HYDRO,FR_NUCLEAR,FR_SOLAR,FR_WINDPOW,FR_RESIDUAL_LOAD,FR_RAIN,FR_WIND,FR_TEMP,GAS_RET,COAL_RET,TARGET
count,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000,851.000000
mean,0.033707,-0.125025,0.125025,-0.033290,0.033290,0.397208,-0.390668,0.163946,-0.332241,0.543592,0.362208,-0.087124,0.023466,0.031605,0.002808,0.039947,0.069333,0.046026
std,0.943071,0.941634,0.941634,1.015605,1.015605,0.925580,0.530913,1.054493,0.989031,1.060339,1.174623,0.922673,1.005274,0.998001,0.979805,1.091868,1.043115,1.023512
min,-1.462350,-2.856874,-2.634831,-2.825331,-1.951516,-1.317350,-0.794950,-1.758523,-2.595326,-1.312588,-1.143277,-1.678936,-1.726420,-1.895319,-5.787097,-5.349463,-5.706442,-6.519268
25%,-0.699925,-0.828152,-0.602534,-0.710545,-0.757525,-0.213153,-0.778677,-0.639208,-0.863334,-0.343903,-0.550885,-0.768812,-0.484767,-0.697246,-0.595664,-0.647117,-0.459531,-0.178023
50%,-0.365601,-0.145508,0.145508,0.041500,-0.041500,0.244309,-0.757349,-0.079222,-0.245237,0.483360,-0.009072,-0.409541,-0.193154,-0.264217,0.008404,0.000000,0.071501,-0.003619
75%,0.739198,0.602534,0.828152,0.757525,0.710545,1.127789,-0.145822,0.852186,0.248058,1.268300,0.930943,0.492683,0.130232,0.491794,0.648230,0.655334,0.647094,0.174344
max,3.300640,2.634831,2.856874,1.951516,2.825331,2.372570,1.905512,3.730478,1.854408,3.648905,4.965793,2.918326,9.473201,4.965028,2.817239,5.674778,3.746576,7.786578


info() permet d'afficher le type de chaque colonnes et si elles sont nulles ou non
describe() permet d'afficher la somme de toutes les lignes de chaque colonne avec "count", la moyenne de la colonne avec "mean", l'écart-type de la colonne avec "std", le minimum et le maximum de chaque colonne avec "min" et "max". De plus, nous avons les 25%, les 50% et les 75% premières valeurs de chaque colonne.


Pour éxaminer la relation entre les variables caractéristiques (chaque colonne) et la variable cible (TARGET) on utilise des techniques graphiques telles que des histogrammes, pour data_DE on n'oublie pas de prendre en compte la colonne en plus dans nos paramètres :

In [5]:
from matplotlib import pyplot as plt
import seaborn as sns

fig, axes = plt.subplots(nrows=6, ncols=3, figsize=(8, 35))
fig.subplots_adjust(hspace=0.5, wspace=0.3)

for i, ax in enumerate(axes.flatten()):
    if i < len(data_FR.describe().columns):
        j = data_FR.describe().columns[i]
        sns.histplot(data_FR[j], ax=ax)
        ax.axvline(data_FR[j].mean(), color='y', linestyle='-', label='Mean')
        ax.axvline(data_FR[j].median(), color='g', linestyle='-', label='Median')
        ax.set_xlabel(j)
        ax.set_title(f'{j} Distribution')
        ax.legend()

plt.tight_layout()
plt.show(block=True)

des diagrammes en boîte :

In [6]:
plt.figure(figsize=(8, 10))
plt.subplots_adjust(hspace=0.5, wspace=0.3)

for i, j in enumerate(data_FR.describe().columns):
    plt.subplot(5, 4, i+1)
    sns.boxplot(x=data_FR[j])
    plt.title('{} Boxplot'.format(j))

plt.tight_layout()
plt.show()

 et des graphiques de dispersion :

In [ ]:
fig, axes = plt.subplots(nrows=6, ncols=3, figsize=(12, 54))
for i, j in enumerate(data_FR.describe().columns):
    axes.flat[i].set_xlabel(j)
    sns.scatterplot(x=data_FR[j], y=data_FR.TARGET, ax=axes.flat[i])
    axes.flat[i].set_title(f'{j} Distribution')

plt.tight_layout()
plt.show(block=True)

In [ ]:
correlation_metrics_1 = data_FR.corr()

fig1 = plt.figure(figsize=(40, 12))
sns.heatmap(correlation_metrics_1, square=True, annot=True, annot_kws={"size": 8}, vmax=1, vmin=-1, cmap='coolwarm')
plt.title('Correlation Between Variables', size=14)
plt.show()